https://huggingface.co/eenzeenee/t5-base-korean-summarization

In [ ]:
!pip install -q wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

summary_data = pd.DataFrame(columns=['summary'])

text_data = pd.read_csv('/content/drive/MyDrive/23-1 데캡디/코드/dataframe_5.csv')
text_data.head(5)

# text_list: 문자열로 변환할 리스트
# separator: 각 문장 사이에 삽입할 구분자 (기본값: 개행 문자)
def join_sentences(text_list, separator="\n"):
    # 리스트의 각 원소를 문자열로 변환
    text_str = separator.join(str(sentence) for sentence in text_list)
    return text_str

# 사용 예시
# text_list = ["첫 번째 문장.", "두 번째 문장.", "세 번째 문장."]
# text_str = join_sentences(text_list, separator="\n")

text_list = text_data['대화'].values.tolist()
print(text_list)

text_str = join_sentences(text_list, separator="\n")

['이 카톡방이 최고인듯합니다! :) 아카이빙을 위해 힘써주시는 분들이 많으니, 참고해보세요!']


In [ ]:
text = text_str

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=100)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)

아카이빙을 위해 힘써주는 사람들이 많으니 참고해 보세요 아카이빙을 위해 힘써주는 사람들이 많으니 참고해 보세요 아카이빙을 위해 힘써주는 사람들이 많으니 참고해 보세요 아카이빙을 위해 힘써주는 사람들이 많으니 참고해


원래 코드드

In [ ]:
import pandas as pd

summary_data = pd.DataFrame(columns=['summary'])

text_data = pd.read_csv('/content/drive/MyDrive/23-1 데캡디/example.csv', usecols=['text'])
text_data.head(5)

,text
0,맞습니다~\n
1,저작권은 그럼 AI한테 있는걸까요?\n
2,아직 저작권은 정해진 바가 없습니다. 현재 OpenAI와 microsoft가 미국에...
3,이게 판결이 나는데도 3-4년 걸릴텐데 그 정도면 …. 소송 전 대비해서 엄청나게 ...
4,사진\n


In [ ]:
import math

# 한 번에 처리할 데이터의 개수
batch_size = 10

# 데이터 개수 계산
num_data = len(text_data)

# 전체 데이터를 batch_size씩 나누어 처리
for i in range(0, num_data, batch_size):
    # 현재 batch에서 처리할 데이터의 범위 계산
    start_idx = i
    end_idx = min(i + batch_size, num_data)
    
    # 현재 batch에서 처리할 데이터 추출
    batch_data = text_data.iloc[start_idx:end_idx]
    #print(batch_data)
    
    # batch_data에서 'text' 열만 추출하여 리스트로 변환
    texts = batch_data['text'].tolist()
    # print(texts)

    string_texts = join_sentences(texts, separator="\n")
    
    # batch_data의 각 행에 대해 요약 수행
    inputs = ["summarize: " + string_texts]

    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
    output = model.generate(**inputs, num_beams=4, do_sample=True, min_length=10, max_length=100)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

    summary_data.loc[i, 'summary'] = predicted_title
    print(predicted_title)
    print(summary_data['summary'].count())
    
    # 현재 batch에서 처리한 데이터 개수 출력
    print(f"Processed {end_idx - start_idx} data.")

# 새로운 DataFrame을 CSV 파일로 저장
summary_data.to_csv('/content/drive/MyDrive/23-1 데캡디/result.csv', index=False)

OpenAI와 microsoft가 미국에서 고소 당한 상태로 소송이 진행 중인데 캐나다에서는 현저한 노력을 해야 저작권을 인정해 주기 때문에 AI가 만들어낸 것들은 누구도 저작권이 인정되기 어려운 상태라고 한다
1
Processed 10 data.
Discord-Zapier-Airtable 연결해서 Midjourney 결과물을 에어테이블에 DB화 시킨거 같다.
2
Processed 10 data.
웨비나 평가 반영해서 웨비나 평가 반영해서 업그레이드 해서 찾아 뵐게요 웨비나 평가 반영해서 웨비나 평가 반영해서 업그레이드 해서 찾아 뵐게요 감사합니다~
3
Processed 10 data.
필터는 에어테이블에서 해도 되고 자피어나 메이크에서 해도 되며 필터는 에어테이블에서 해도 되고 자피어나 메이크에서 해도 된다.
4
Processed 10 data.
ChatGPT 스터디를 기획해 보았는데 공지 드래프트를 보고 피드백을 해 줄 수 있는 사람이 있다면 톡을 부탁한다.
5
Processed 10 data.
스터디를 ChatGPT로 흡수해서 하려고 하는데 민트베어에게 피드백을 받아야 할 것 같다.
6
Processed 10 data.
ChatGPT로 사업하는데 관심 있어서 들어왔는데 세미나 다시 듣기도 가능한가요?
7
Processed 10 data.
직접 이미지를 생성하지 않고 이미 생성된 것을 검색하는 엔진은 블로그 글에서 쓸 이미지 정도가 필요해서 유용할 것 같다.
8
Processed 10 data.
상업적 사용이 가능한가요?
9
Processed 10 data.
리서치하는 입장에서도 어마어마해서 리서치에 활용할 때 인사이트를 줄 만한 답이 나오나 하는 질문이나 리서치에 활용할 때 인사이트를 줄 만한 답이 나오나 하는 의문이 든다.
10
Processed 10 data.
대체투자는 주식, 채권 외의 모든 투자이며 주력은 부동산, 신재생이고 삭제된 메시지로 대체투자는 주식, 채권 외의 모든 투자이며 주력은 부동산, 신재생이다.
11
Processed 10 data

KeyboardInterrupt: ignored

In [ ]:
summary_data.to_csv('/content/drive/MyDrive/23-1 데캡디/result.csv', index=False)